# Спец. розділи програмування
## Наука про дані: підготовчий етап
### Виконав: Щупаківський Роман, Фі-31

#### Імпорт бібліотек:

In [4]:
#Завантажуємо бібліотеки
import pandas as pd
from urllib import request
import os
from shutil import rmtree
from datetime import datetime
import re

In [5]:
#Функція для перевірки чи пуста папка
def check(folder_path):
    if os.listdir(folder_path):
        rmtree(folder_path)
        os.makedirs(folder_path)

In [6]:
#
def get_data(p_id):
    url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={p_id}&year1=1981&year2=2024&type=Mean'

    vhi = request.urlopen(url).read().decode('utf-8')
    now = datetime.now()
    f_time = now.strftime('%d%m%Y%H%M%S')
    with open(f'data/vhi_id_{p_id}_{f_time}.csv', 'w') as f:
        f.write(vhi)

In [10]:
check('data')
for i in range(1, 28):
    get_data(i)

In [60]:
#створення дф, підготовка його
def create_dataframe(csv_path, p_id):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(csv_path, header=1, names=headers)
    df = df[df['VHI'] != -1]
    df.at[0, 'Year'] = df.at[0, 'Year'][9:]
    df = df.drop(columns='empty').drop(df.index[-1])
    df['area'] = p_id
    df['Year'] = df['Year'].astype(int)
    return df

In [61]:
#Збирає всі шляхи до файлів в папці дата
def paths_list():
    file_paths = []
    for root, dirs, files in os.walk('data'):
        for i in files:
            file_paths.append(os.path.join(root, i))

    return file_paths

In [25]:
#Вилучає ідентифікатор провінції
def get_p_id(file_path):
    with open(file_path, 'r') as file:
        first_line = file.readline()
        match = re.search(r'Province\s*=\s*(\d+)', first_line)
        return int(match.group(1))


#Заміна індексів
provinces_dict = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4,
                  7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 12: 9,
                  13: 10, 14: 11, 15: 12, 16: 13, 17: 14,
                  18: 15, 19: 16, 20: 25, 21: 17, 22: 18,
                  23: 6, 24: 1, 25: 2, 26: 7, 27: 5}

#Створення дф і об'єднання їх у мейн
main_df = pd.DataFrame()
for file_path in paths_list():
    p_id = provinces_dict[get_p_id(file_path)]
    df = create_dataframe(file_path, p_id)
    main_df = pd.concat([main_df, df], ignore_index=True)

In [63]:
main_df.tail()

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
58720,2024,37.0,0.172,302.17,26.35,8.43,17.41,20
58721,2024,38.0,0.166,300.82,27.57,6.46,17.03,20
58722,2024,39.0,0.160,299.27,29.74,4.25,17.01,20
58723,2024,40.0,0.157,297.55,32.72,2.74,17.73,20
58724,2024,41.0,0.155,295.40,36.69,1.18,18.94,20


### Ряд VHI для області за вказаний рік:

In [64]:
def task1(province_id, year):
    new_df = main_df.loc[(main_df['area'] == province_id) & (main_df['Year'] == year)]
    ryad = pd.Series(new_df['VHI']).reset_index(drop=True)
    return ryad

task1(1, 2000)

0     24.22
1     27.70
2     30.68
3     32.55
4     34.73
5     35.08
6     33.79
7     34.60
8     37.70
9     38.67
10    38.05
11    39.32
12    40.21
13    41.25
14    43.66
15    47.61
16    52.54
17    60.14
18    62.91
19    63.27
20    61.76
21    58.32
22    56.10
23    55.32
24    55.03
25    54.68
26    55.27
27    56.28
28    57.93
29    57.94
30    57.29
31    56.45
32    54.95
33    55.00
34    55.77
35    54.21
36    54.28
37    52.57
38    48.01
39    40.57
40    33.71
41    29.41
42    26.21
43    18.19
44    12.26
45    11.28
46    11.25
47    11.38
48    12.91
49    14.20
50    15.07
51    18.89
Name: VHI, dtype: float64

### Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани:

In [65]:
def task2(p_ids, years):
    filtered_df = main_df.loc[
        (main_df['area'].isin(p_ids)) & (main_df['Year'].isin(years)), ['area', 'Year', 'VHI']]

    mean_vhi = main_df.groupby(['Year', 'area'])['VHI'].mean().reset_index(name='mean_VHI')
    min_vhi = main_df.groupby(['Year', 'area'])['VHI'].min().reset_index(name='min_VHI')
    max_vhi = main_df.groupby(['Year', 'area'])['VHI'].max().reset_index(name='max_VHI')
    median_vhi = main_df.groupby(['Year', 'area'])['VHI'].median().reset_index(name='median_VHI')

    stats_df = mean_vhi.merge(min_vhi, on=['Year', 'area'])
    stats_df = stats_df.merge(max_vhi, on=['Year', 'area'])
    stats_df = stats_df.merge(median_vhi, on=['Year', 'area'])

    merged_df = filtered_df.merge(stats_df, on=['Year', 'area'])
    merged_df = merged_df.drop(['VHI'], axis=1)
    merged_df = merged_df.drop_duplicates().reset_index(drop=True)
    return merged_df

task2([2,4,8], [2010,2014,2018])

,area,Year,mean_VHI,min_VHI,max_VHI,median_VHI
0,2,2010,49.635769,38.35,62.11,48.695
1,2,2014,52.470192,44.16,66.15,50.995
2,2,2018,49.662500,38.30,57.03,50.340
3,4,2010,44.960385,32.63,58.36,46.705
4,4,2014,46.418654,32.88,66.24,44.705
5,4,2018,39.644231,27.64,49.28,41.255
6,8,2010,50.591154,39.32,64.46,49.715
7,8,2014,51.415769,40.53,64.62,50.320
8,8,2018,49.923462,31.99,61.12,50.310


### Ряд VHI за вказаний діапазон років для вказаних областей:

In [66]:
def task3(province_ids, year1, year2):
    new_df = main_df.loc[
        (main_df['area'].isin(province_ids)) & (main_df['Year'] >= year1) & (main_df['Year'] <= year2)]
    new_df = new_df[['Year', 'Week', 'VHI', 'area']]
    new_df = new_df.drop_duplicates().reset_index(drop=True)
    return new_df

task3([2,4,8], 2000, 2004)

,Year,Week,VHI,area
0,2000,1.0,24.65,2
1,2000,2.0,27.49,2
2,2000,3.0,31.36,2
3,2000,4.0,37.28,2
4,2000,5.0,40.85,2
...,...,...,...,...
718,2004,48.0,51.84,8
719,2004,49.0,51.67,8
720,2004,50.0,50.15,8
721,2004,51.0,50.84,8


### Bиявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей. Повернути роки, назви областей та значення VHI:

In [67]:
def task4(percent):
    new_df = main_df[['Year', 'area', 'VHI']].copy()
    new_df = new_df[new_df['VHI'] < 15]
    da = new_df.groupby('Year')['area'].nunique().reset_index()
    da.columns = ['Year', 'Amount']
    provinces_amount = round(27 * (percent / 100))
    drought_years = da[da['Amount'] > provinces_amount]
    final_df = new_df[new_df['Year'].isin(drought_years['Year'])]

    return final_df

task4(10)

,Year,area,VHI
3124,2000,9,12.51
3125,2000,9,10.60
3126,2000,9,11.20
3127,2000,9,12.32
3128,2000,9,14.65
...,...,...,...
57828,2007,20,12.96
57829,2007,20,13.48
57830,2007,20,14.05
57831,2007,20,14.41
